In [1]:
import json
import numpy as np
import pandas as pd
import torch

from torch.nn import Linear, LSTM, Tanh, ReLU, Module, MSELoss
from torchvision import transforms
from pprint import pprint

from src.speech_classification.audio_processing import AudioPreprocessor
from src.speech_classification.pytorch_conv_lstm import LstmNet
from src.reinforcement.model_dynamics.rnn_md import LstmModelDynamics
from src.VTL.utils import random_rollout

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 1 - Load config and specify model dynamics net checkpoint to load from

In [ ]:
with open('rnn_md_config.json') as data_file:
    params = json.load(data_file)
md_net_fname=''

## 2 - Load model

In [ ]:
print(params)

device = params['train']['device']

# 1. Init audio preprocessing
preproc = AudioPreprocessor(**params['preprocessing_params'])
sr = params['preprocessing_params']['sample_rate']

# 2. Load preprocessing net
preproc_net = torch.load(params['preproc_net_fname']).to(device)

# 3. Init model dynamics net
md_net = torch.load(md_net_fname).to(device)

## 3 - Generate sample

In [ ]:
speaker_fname = os.path.join(os.path.dirname(os.path.abspath(__file__)), 'JD2.speaker')
lib_path = os.path.join(os.path.dirname(os.path.abspath(__file__)), 'VocalTractLab2.dll')
num_episodes = 10
ep_duration = 1000
timestep = 20
num_steps_per_ep = ep_duration // timestep

env = VTLEnv(lib_path, speaker_fname, timestep, max_episode_duration=ep_duration)

soundnames = ['a', 'i', 'o', 'u']
initial_states = [env.get_cf(s) for s in soundnames]
audios, states, actions = random_rollout(env, ep_duration, timestep, state)

## 4 - Predict and Plot